<a href="https://colab.research.google.com/github/paulyu8868/test/blob/main/%EB%96%A8%EC%82%AC%EC%98%A4%ED%8C%94_%EB%B0%B1%ED%85%8C%EC%8A%A4%ED%8A%B8(%EC%B5%9C%EC%A2%85%EB%B3%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass


import warnings
# 모든 FutureWarning 무시
warnings.filterwarnings('ignore', category=FutureWarning)
# UserWarning도 무시
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
#기간별 주가 데이터 불러오기
def get_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    df = df.drop(columns=['Volume', 'Adj Close'])
    df.index = df.index.date

    # 호가 단위 0.01$ 적용
    for col in ['Open', 'High', 'Low', 'Close']: # 시가,고가,저가,종가
        df[col] = df[col].map(round_half_up_to_two)

    # 등락율 계산
    df['Return'] = df['Close'].pct_change()
    df['Return'] = df['Return'].map(pointTopercent)
    #df['Return'] = df['Return'].map(round_half_up_to_two)

    return df

In [3]:
# 소수 셋째자리에서 반올림
def round_half_up_to_two(num):
    try:
        if isinstance(num, (float, int)):
            num_100 = num * 100
            if num_100 - int(num_100) >= 0.5:
                return (int(num_100) + 1) / 100
            else:
                return int(num_100) / 100
        else:
            return num
    except:
        return num

# 퍼센트로 변환
def pointTopercent(num):
    return round_half_up_to_two(num*100)

In [4]:
def calculate_mdd(equity_curve):
    """MDD(Maximum Drawdown) 계산"""
    cummax = equity_curve.cummax()
    drawdown = (equity_curve - cummax) / cummax * 100
    mdd = drawdown.min()
    return mdd

In [19]:
# 떨사오팔 매매 로직
def infinite_buy_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period,fee,welfare):
    funds = initial_funds # 초기 자금
    one_buy_amount = initial_funds / buy_portion # 회차별 매수금액
    holdings = 0 # 보유 주식 수
    buy_records = []  # 각 매수 건을 저장하여 관리 (개별 매도 관리)
    trade_history = []  # 매도 기록 저장
    trade_id = 1  # 매수 회차별 ID (매수 구분)
    fee = (fee/100) # 수수료

    # 날짜별 거래기록 데이터 프레임
    df_res = pd.DataFrame(index=range(start_idx, start_idx + simulation_period + 1),
                         columns=['날짜', '시가', '고가', '종가', '등락율',
                                'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                '보유 주식 수', '예수금', '총 평가액', '수익율(%)'])

    # 시작일 - 종료일 시뮬레이션 진행
    for i in range(start_idx, start_idx + simulation_period + 1):
        # 데이터 시리즈로 가져오기
        current_date = df.index[i]
        open_price = float(df['Open'].iloc[i]) #시가
        high_price = float(df['High'].iloc[i]) #고가
        close_price = float(df['Close'].iloc[i]) #종가
        return_val = float(df['Return'].iloc[i]) #등락율
        prev_price = float(df['Close'].iloc[i-1]) if i > 0 else close_price #전날 종가

        # 오늘 데이터
        df_res.at[i, '날짜'] = current_date
        df_res.at[i, '시가'] = open_price
        df_res.at[i, '고가'] = high_price
        df_res.at[i, '종가'] = close_price
        df_res.at[i, '등락율'] = f"{round_half_up_to_two(return_val)}%"

        # 당일 종가
        price = close_price

        # 매수/매도 칼럼 초기화
        df_res.at[i, 'LOC 매수'] = 0
        df_res.at[i, '수익 실현 매도'] = 0
        df_res.at[i, 'MOC 손절'] = 0

        # 복리 투자 시
        T = len(buy_records)
        one_buy_welfare = funds / (buy_portion-T) if buy_portion>T else funds # 복리투자 1회차 금액

        # 매수 로직
        if price <= prev_price: # 전날 종가 LOC 매수
            if welfare: # 복리 적용
                qty = int(one_buy_welfare / prev_price)
            else: # 단리 적용
                qty = int(one_buy_amount / prev_price)
            if funds >= (qty * price) * (1+fee): # 수수료 적용한 금액이상이 남아있을때 매수
                holdings += qty
                funds -= (qty * price) * (1+fee) # 수수료 차감
                buy_records.append({
                    'id': trade_id,
                    'buy_date': current_date,
                    'buy_price': price,
                    'quantity': qty,
                    'days': 0,
                    'type': 'LOC 매수'
                })
                df_res.at[i, 'LOC 매수'] = qty
                trade_id += 1

        # 매도 로직
        if holdings > 0:
            new_buy_records = []  # 매도되지 않은 매수 건을 저장할 새 리스트
            total_sell = 0  # 당일 총 매도 수량

            for record in buy_records: # 보유 주식 순회
                record['days'] += 1  # 보유일 count
                # 각 매수 건별 매도 목표가
                #sell_price = record['buy_price'] * 1.005 # 한국투자증권 온라인 수수료 * 2
                sell_price = record['buy_price'] * (1+fee*2)  # 백테스트 기준 0.044% * 2

                if price >= sell_price:  # 수익 실현 매도 조건 충족
                    funds += record['quantity'] * price
                    funds -= (record['quantity'] * price)*fee # 수수료 차감
                    total_sell += record['quantity']
                    holdings -= record['quantity']
                    # 거래 기록 저장
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round_half_up_to_two((price/record['buy_price'] - 1) * 100)
                    })
                elif record['days'] >= 41:  # 30일 경과 시 손절
                    funds += record['quantity'] * price
                    funds -= (record['quantity'] * price)*fee # 수수료 차감
                    df_res.at[i, 'MOC 손절'] = record['quantity']
                    holdings -= record['quantity']
                    # 손절 거래 기록 저장
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': (price/record['buy_price'] - 1) * 100
                    })
                else:
                    new_buy_records.append(record)

            if total_sell > 0:
                df_res.at[i, '수익 실현 매도'] = total_sell

            buy_records = new_buy_records

        # 포트폴리오 상태 저장
        df_res.at[i, '보유 주식 수'] = holdings
        df_res.at[i, '예수금'] = funds
        df_res.at[i, '총 평가액'] = funds + (price * holdings)
        df_res.at[i, '수익율(%)'] = ((funds + (price * holdings)) / initial_funds - 1) * 100
        df_res.at[i, 'MDD'] = calculate_mdd(df_res['총 평가액'])

    final_value = funds + (holdings * float(df['Close'].iloc[start_idx + simulation_period]))
    return round_half_up_to_two((final_value / initial_funds - 1) * 100), df_res, final_value, pd.DataFrame(trade_history)

In [20]:
if __name__ == "__main__":
    start_date = '2024-01-01'
    end_date = '2024-12-31'
    initial_funds = 100000 # 초기 자금
    buy_portion = 8  # 회차 분할 수
    fee = 0.09 # 수수료(%)
    stock_item = 'SOXL'
    welfare = False
    welfare = True

    # 시작일 30일 전의 날짜
    start_date_dt = datetime.strptime(start_date,'%Y-%m-%d')
    start_date_before_30 = start_date_dt - timedelta(days=30)
    start_date_before_30 = start_date_before_30.strftime('%Y-%m-%d')

    # 종료일 다음 날 계산
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
    next_day = end_date_dt + timedelta(days=1)
    end_day_next = next_day.strftime('%Y-%m-%d')

    # 30일 전 데이터부터 입력
    # ticker 값으로 종목 선택 가능 ex) TQQQ
    df = get_data(ticker=stock_item, start=start_date_before_30, end=end_day_next)
    df_length = len(df) - len(get_data(ticker=stock_item, start=start_date, end=end_day_next))

    df_res = pd.DataFrame(columns=['날짜', '시가', '고가', '종가', '등락율', 'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                  '보유 주식 수', '예수금', '총 평가액', '수익율(%)', 'MDD'])

    # 시뮬레이션 실행
    return_rate, df_res, final_value, df_trades = infinite_buy_simulation(
        df, df_res, initial_funds, buy_portion, df_length, len(df)-1-df_length,fee, welfare)


    # 매매 통계 출력
    print('\n' + '='*80)
    print("매매 통계")
    print('='*80)
    print(f"총 매매 횟수: {len(df_trades)} 회")
    print(f"평균 보유기간: {df_trades['보유기간'].mean():.1f} 일")
    print(f"평균 수익률: {df_trades['수익률(%)'].mean():.2f}%")
    win_rate = len(df_trades[df_trades['수익률(%)'] > 0]) / len(df_trades) * 100
    print(f"승률: {win_rate:.2f}%")

    print('\n' + '='*80)
    print(f"{start_date} ~ {end_date} 동안의 자산 변동 결과")
    print('='*80)
    print(f"최초 보유 금액: ${initial_funds:,.2f}")
    print(f"최종 보유 금액: ${final_value:,.2f}")
    print(f"원금 변화율: {return_rate}%")
    print('='*80)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



매매 통계
총 매매 횟수: 117 회
평균 보유기간: 4.0 일
평균 수익률: 4.34%
승률: 99.15%

2024-01-01 ~ 2024-12-31 동안의 자산 변동 결과
최초 보유 금액: $100,000.00
최종 보유 금액: $169,887.81
원금 변화율: 69.89%


In [ ]:
# 날짜별 거래 현황 출력
print("\n[일별 거래 현황]")
df_res_style = df_res.style.format({
    '시가': '{:.2f}',
    '고가': '{:.2f}',
    '종가': '{:.2f}',
    'LOC 매수': '{:.0f}',
    '수익 실현 매도': '{:.0f}',
    'MOC 손절': '{:.0f}',
    '보유 주식 수': '{:.0f}',
    '예수금': '{:.2f}',
    '총 평가액': '{:.2f}'
}).set_properties(**{'text-align': 'center'})
display(df_res_style)


[일별 거래 현황]


,날짜,시가,고가,종가,등락율,LOC 매수,수익 실현 매도,MOC 손절,보유 주식 수,예수금,총 평가액,수익율(%),MDD
21,2020-01-02,22.18,22.71,22.71,4.94%,0,0,0,0,100000.00,100000.00,0.000000,0.000000
22,2020-01-03,21.82,22.50,22.09,-2.73%,550,0,0,550,87845.15,99994.65,-0.005346,-0.005346
23,2020-01-06,21.57,22.52,22.51,1.9%,0,0,0,550,87845.15,100225.65,0.225654,-0.005346
24,2020-01-07,22.53,22.69,22.49,-0.08%,555,0,0,1105,75357.71,100209.16,0.209162,-0.016455
25,2020-01-08,22.49,23.32,23.00,2.27%,0,0,0,1105,75357.71,100772.71,0.772712,-0.016455
26,2020-01-09,23.56,23.72,23.58,2.52%,0,0,0,1105,75357.71,101413.61,1.413612,-0.016455
27,2020-01-10,23.85,23.87,23.39,-0.8%,530,0,0,1635,62955.56,101198.21,1.198208,-0.212402
28,2020-01-13,23.68,24.22,24.21,3.51%,0,0,0,1635,62955.56,102538.91,2.538908,-0.212402
29,2020-01-14,24.15,24.29,23.92,-1.19%,516,0,0,2151,50607.41,102059.33,2.059327,-0.467706
30,2020-01-15,23.94,24.28,23.94,0.08%,0,0,0,2151,50607.41,102102.35,2.102347,-0.467706


In [ ]:
# 모든 매매 로그 출력
if not df_trades.empty:
    print("\n[매매 기록]")
    df_trades_style = df_trades.style.format({
        '매수가': '{:.2f}',
        '매수수량': '{:.0f}',
        '매도가': '{:.2f}',
        '매도수량': '{:.0f}',
        '수익률(%)': '{:.2f}'
    }).set_properties(**{'text-align': 'center'})

    # 수익률에 따른 색상 적용
    def color_returns(val):
        color = 'red' if val < 0 else ('blue' if val > 0 else 'black')
        return f'color: {color}'

    df_trades_style = df_trades_style.map(color_returns, subset=['수익률(%)'])
    display(df_trades_style)




[매매 기록]


,회차,매수일,매수가,매수수량,매도일,매도가,매도수량,보유기간,수익률(%)
0,1,2020-02-07,26.75,461,2020-02-10,27.72,461,2,3.63
1,2,2020-02-13,28.44,437,2020-02-14,28.64,437,2,0.70
2,6,2020-02-25,21.99,522,2020-02-26,22.29,522,2,1.36
3,7,2020-02-27,18.95,560,2020-02-28,19.25,560,2,1.58
4,8,2020-03-03,19.82,571,2020-03-04,22.22,571,2,12.11
5,11,2020-03-09,15.31,650,2020-03-10,17.69,650,2,15.55
6,13,2020-03-12,11.19,811,2020-03-13,14.21,811,2,26.99
7,14,2020-03-16,9.31,879,2020-03-17,10.83,879,2,16.33
8,15,2020-03-18,9.83,1154,2020-03-19,10.06,1154,2,2.34
9,16,2020-03-20,8.88,1242,2020-03-23,8.90,1242,2,0.23



매매 통계
총 매매 횟수: 39 회
평균 보유기간: 7.8 일
평균 수익률: 1.59%
승률: 87.18%

2020-02-01 ~ 2020-06-30 동안의 자산 변동 결과
최초 보유 금액: $100,000.00
최종 보유 금액: $105,991.54
원금 변화율: 5.99%


In [ ]:
# 기간별 성과 분석 프레임
def analyze_performance_by_period(df_trades, period='M'):
    """
    period options:
    - 'M': 월별
    - 'Q': 분기별
    - 'Y': 연도별
    """
    # 매수일을 datetime으로 변환
    df_trades['매수일'] = pd.to_datetime(df_trades['매수일'])

    # 기간별 그룹화
    df_trades['period'] = df_trades['매수일'].dt.to_period(period)

    # 기간별 성과 계산
    performance = []
    for period, group in df_trades.groupby('period'):
        profit_trades = group[group['수익률(%)'] > 0]
        loss_trades = group[group['수익률(%)'] <= 0]

        performance.append({
            '기간': period.strftime('%Y-%m'),
            '거래횟수': len(group),
            '승률(%)': round(len(profit_trades) / len(group) * 100, 2),
            '평균수익률(%)': round(profit_trades['수익률(%)'].mean() if len(profit_trades) > 0 else 0, 2),
            '평균손실률(%)': round(loss_trades['수익률(%)'].mean() if len(loss_trades) > 0 else 0, 2),
            '최대수익률(%)': round(group['수익률(%)'].max(), 2),
            '최대손실률(%)': round(group['수익률(%)'].min(), 2),
            '평균보유기간': round(group['보유기간'].mean(), 1)
        })

    df_performance = pd.DataFrame(performance)

    # 수익, 손실 색상 구분
    def style_percentage(val):
        try:
            num = float(str(val).replace('%', ''))
            color = 'red' if num < 0 else ('blue' if num > 0 else 'black')
            return f'color: {color}'
        except:
            return 'color: black'

    df_performance_style = df_performance.style.format({
        '승률(%)': '{:.2f}',
        '평균수익률(%)': '{:.2f}',
        '평균손실률(%)': '{:.2f}',
        '최대수익률(%)': '{:.2f}',
        '최대손실률(%)': '{:.2f}',
        '평균보유기간': '{:.1f}'
    }).set_properties(**{
        'text-align': 'center',
        'font-family': 'NanumGothic',
        'width': '100px'
    }).map(style_percentage, subset=[
        '평균수익률(%)',
        '평균손실률(%)',
        '최대수익률(%)',
        '최대손실률(%)'
    ])

    return df_performance_style



In [ ]:
# # 기간별 분석 출력 (월별)
# print("\n[월별 성과 분석]")
# monthly_performance = analyze_performance_by_period(df_trades, 'M')
# display(monthly_performance)

# # 기간별 분석 출력 (분기별)
# print("\n[분기별 성과 분석]")
# quarterly_performance = analyze_performance_by_period(df_trades, 'Q')
# display(quarterly_performance)

# 기간별 분석 출력 (연도별)
print("\n[연도별 성과 분석]")
yearly_performance = analyze_performance_by_period(df_trades, 'Y')
display(yearly_performance)

# 전체 통계 요약
print("\n[전체 기간 통계 요약]")
print("="*80)
profit_trades = df_trades[df_trades['수익률(%)'] > 0]
loss_trades = df_trades[df_trades['수익률(%)'] <= 0]

print(f"총 거래 횟수: {len(df_trades)} 회")
print(f"전체 승률: {len(profit_trades)/len(df_trades)*100:.2f}%")
print(f"평균 수익 (이익 거래): {profit_trades['수익률(%)'].mean():.2f}%")
print(f"평균 손실 (손실 거래): {loss_trades['수익률(%)'].mean():.2f}%")
print(f"최대 수익률: {df_trades['수익률(%)'].max():.2f}%")
print(f"최대 손실률: {df_trades['수익률(%)'].min():.2f}%")
print(f"평균 보유기간: {df_trades['보유기간'].mean():.1f}일")
print(f"수익 대 손실 비율: {abs(profit_trades['수익률(%)'].mean()/loss_trades['수익률(%)'].mean()):.2f}")
print("="*80)


[연도별 성과 분석]


,기간,거래횟수,승률(%),평균수익률(%),평균손실률(%),최대수익률(%),최대손실률(%),평균보유기간
0,2015-12,102,85.29,3.49,-27.93,13.24,-44.62,8.2
1,2016-12,99,97.98,3.06,-25.90,12.30,-29.65,5.1
2,2017-12,98,100.00,2.73,0.00,8.27,0.59,4.4
3,2018-12,114,87.72,3.87,-20.02,17.23,-36.43,8.2
4,2019-12,107,95.33,4.31,-19.58,17.15,-35.88,5.7
5,2020-12,118,94.92,6.10,-50.70,30.51,-74.51,5.4
6,2021-12,118,97.46,4.86,-13.74,17.91,-22.12,5.4
7,2022-12,132,87.88,6.21,-30.96,30.74,-53.68,7.8
8,2023-12,114,95.61,4.60,-16.03,19.46,-20.56,5.7



[전체 기간 통계 요약]
총 거래 횟수: 1002 회
전체 승률: 93.41%
평균 수익 (이익 거래): 4.44%
평균 손실 (손실 거래): -26.82%
최대 수익률: 30.74%
최대 손실률: -74.51%
평균 보유기간: 6.3일
수익 대 손실 비율: 0.17
